In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import OUTPUT_PATH, Version, FuncGroups, CollisionType


def count_dup(v: Version):
    functions = FuncGroups.from_jsonl(v.funcs_path)
    results = {c: FuncGroups() for c in CollisionType}

    for name, group in functions.data:
        if len(group) == 1:
            results[CollisionType.UNIQUE].add_group(name, group)
            continue

        # Those are the functions that are declared in single header
        # but defined in multiple files
        if len({func.loc for func in group}) == 1:
            results[CollisionType.HEADER].add_group(name, group)
            continue

        # External functions have name collision with static functions
        if any(func.external for func in group):
            results[CollisionType.MIXED].add_group(name, group)
            continue

        # Static functions have name collision with other static functions
        results[CollisionType.STATIC].add_group(name, group)

    for c in [CollisionType.HEADER, CollisionType.STATIC, CollisionType.MIXED]:
        results[c].save_result(OUTPUT_PATH / "dup" / v.name / f"{c.name.lower()}.txt")

    return {k: v.num_funcs for k, v in results.items()}


# count_dup(VERSION_DEFAULT)

In [3]:
from depsurf import VersionGroup
from depsurf.output import save_df

df = VersionGroup.apply(count_dup)
save_df(df, "dup")

[    utils.py:160] INFO: NumExpr defaulting to 8 threads.
[func_groups.py:33 ] INFO: Saved 4042 groups to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/header.txt
[func_groups.py:33 ] INFO: Saved 404 groups to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/static.txt
[func_groups.py:33 ] INFO: Saved 10 groups to /Users/szhong/Downloads/bpf-study/output/dup/4.4.0-21-generic-amd64/mixed.txt
[func_groups.py:33 ] INFO: Saved 4329 groups to /Users/szhong/Downloads/bpf-study/output/dup/4.8.0-22-generic-amd64/header.txt
[func_groups.py:33 ] INFO: Saved 401 groups to /Users/szhong/Downloads/bpf-study/output/dup/4.8.0-22-generic-amd64/static.txt
[func_groups.py:33 ] INFO: Saved 26 groups to /Users/szhong/Downloads/bpf-study/output/dup/4.8.0-22-generic-amd64/mixed.txt
[func_groups.py:33 ] INFO: Saved 4430 groups to /Users/szhong/Downloads/bpf-study/output/dup/4.10.0-19-generic-amd64/header.txt
[func_groups.py:33 ] INFO: Saved 397 groups to /Users/szhong

Unique  Include Duplication  Static-Static Collision  Static-Global Collision
reg    4.4.0-21-generic-amd64      52947                53294                      961                       20
       4.8.0-22-generic-amd64      55625                58314                      971                       52
       4.10.0-19-generic-amd64     57185                61008                      963                       48
       4.13.0-16-generic-amd64     59707                63567                      972                       50
       4.15.0-20-generic-amd64     61838                65765                      975                       52
       4.18.0-10-generic-amd64     65198                71380                     1012                       52
       5.0.0-13-generic-amd64      66721                74093                     1010                       52
       5.3.0-18-generic-amd64      69291                79741                     1019                       52
       5.4.0-26-generic-amd64      70906                81585                     1033                       54
       5.8.0-25-generic-amd64      73473                87832                     1061                       52
       5.11.0-16-generic-amd64     77519                92074                     1105                       52
       5.13.0-19-generic-amd64     78691                94698                     1117                       52
       5.15.0-25-generic-amd64     79973                96493                     1145                       52
       5.19.0-21-generic-amd64     82519                98428                     1176                       58
       6.2.0-20-generic-amd64      84682                99613                     1191                       64
       6.5.0-9-generic-amd64       87953               105155                     1255                       61
       6.8.0-22-generic-amd64      91713               112083                     1266                       60
arch   5.4.0-26-generic-arm64      72704               119398                     1165                       54
       5.4.0-26-generic-armhf      70568                76106                     1039                       70
       5.4.0-26-generic-ppc64el    62914                72195                      904                       48
flavor 5.4.0-26-lowlatency-amd64   70953                79123                     1037                       54
       5.4.0-1009-aws-amd64        68965                79546                      997                       52
       5.4.0-1009-gcp-amd64        71068                82436                     1056                       52
       5.4.0-1010-azure-amd64      67581                79154                      901                       48